In [4]:
import pandas as pd

lexique = pd.read_csv('Lexique383/Lexique383.tsv', sep='\t')

In [5]:
lexique.head(5)

,ortho,phon,lemme,cgram,genre,nombre,freqlemfilms2,freqlemlivres,freqfilms2,freqlivres,...,orthrenv,phonrenv,orthosyll,cgramortho,deflem,defobs,old20,pld20,morphoder,nbmorph
0,a,a,a,NOM,m,NaN,81.36,58.65,81.36,58.65,...,a,a,a,"NOM,AUX,VER",NaN,NaN,1.00,1.00,a,1
1,a,a,avoir,AUX,NaN,NaN,18559.22,12800.81,6350.91,2926.69,...,a,a,a,"NOM,AUX,VER",NaN,NaN,1.00,1.00,avoir,1
2,a,a,avoir,VER,NaN,NaN,13572.40,6426.49,5498.34,1669.39,...,a,a,a,"NOM,AUX,VER",93.0,16.0,1.00,1.00,avoir,1
3,a capella,akapEla,a capella,ADV,NaN,NaN,0.04,0.07,0.04,0.07,...,allepac a,alEpaka,a ca-pel-la,ADV,NaN,NaN,3.85,2.85,a-capella,2
4,a cappella,akapEla,a cappella,ADV,NaN,NaN,0.04,0.07,0.04,0.07,...,alleppac a,alEpaka,a cap-pel-la,ADV,NaN,NaN,4.60,2.85,a-cappella,2


In [6]:
lexiqueClean = {}

for i, row in lexique.iterrows():
    

    print(i, row['ortho'])
    break

0 a


In [7]:
def cosine(t1, t2):
    return (torch.nn.functional.normalize(t1)*torch.nn.functional.normalize(t2)).sum(-1)

In [8]:
import torch
camembert = torch.hub.load('pytorch/fairseq', 'camembert')
camembert.eval()  # disable dropout (or leave in train mode to finetune)

Using cache found in /home/hoomano/.cache/torch/hub/pytorch_fairseq_main


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


2022-06-29 12:19:47 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/camembert-base.tar.gz from cache at /home/hoomano/.cache/torch/pytorch_fairseq/3a5b985f6506f03df591fc4564446205fffa6bfe1a5d4657775ed20efdf3162b.4f0d00e7c3a06d48868273ae5e32b16aeb9cafa08939dd691352a8bdae6059be
2022-06-29 12:19:49 | INFO | fairseq.tasks.masked_lm | dictionary: 32004 types


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(32005, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout_module

In [14]:
line = "J'aime le camembert !"
tokens = camembert.encode(line)
last_layer_features = camembert.extract_features(tokens)

In [27]:
a

10

In [15]:
last_layer_features.shape

torch.Size([1, 10, 768])

In [24]:
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

sentences = """Je fais souvent ce rêve étrange et pénétrant
Je fais souvent ce rêve d'avoir l'biff de The Rock
J'veux l'biff comme The Rock""".split('\n')

embeddings = model.encode(sentences)

No sentence-transformers model found with name /home/hoomano/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


In [25]:
import torch
torchEmb = torch.nn.functional.normalize(torch.from_numpy(embeddings))

(torchEmb@torchEmb.T)

tensor([[1.0000, 0.4999, 0.2042],
        [0.4999, 1.0000, 0.7509],
        [0.2042, 0.7509, 1.0000]])

In [39]:
targetSentences = ["Je fais souvent ce rêve étrange et pénétrant", "J'veux l'biff comme The Rock"]
targetWeights = torch.Tensor([1., 1.])
targetWeights /= targetWeights.sum()


def getFitness(sentence):
    embeddings = model.encode([sentence] + targetSentences)

    norm_embeddings = torch.nn.functional.normalize(torch.from_numpy(embeddings))
    return (((norm_embeddings@norm_embeddings.T)[0, 1:] * targetWeights)**0.5).sum()

print(getFitness("Je fais souvent ce rêve d'avoir l'biff de The Rock"))
print(getFitness("Je fais souvent ce rêve étrange et pénétrant"))
print(getFitness("J'veux l'biff comme The Rock"))

tensor(1.1127)
tensor(1.0266)
tensor(1.0266)


In [61]:
import random as rd
import copy as cp

rawCorpus = ' '.join("""Freeze Raël, sur la prod, kicke comme Israel
Fuck ces nègres comme Israël
Chen Laden dans l'complot comme les Ben Laden, Chirak comme JB Binladen
J'les vois petit comme si j'étais 100 pieds, on veut les VVS sur les dentiers
À propos des sommes comme les banquiers, investisseurs, architectes et chefs de chantier
J'veux l'biff comme The Rock
20-20, le rap, c'est satanique comme le rock
Cali Weed, Promethazine et l'fer comme D-Block Europe
Concurrence j'laisse que des mégots
J'suis là pour les deux peines de mort pour les pédos
J'mixe le lin dans les Faygo, à Dakar, on empile les briques comme des Lego""".split('\n'))
rawCorpus = ""
with open("./corpus.txt") as f:
    rawCorpus = ' '.join(f.readlines())

corpus = [x for x in ' '.join(rawCorpus.split('\'')).split(' ') if x.isalpha()]

class Agent:

    def __init__(self):
        self.fitness = 0
        self.sentence = "Je fais souvent ce rêve étrange et pénétrant".split(' ')  # []  # [int]

    def getFitness(self):
        self.fitness = getFitness(self.sentenceToStr()) # ToDo

    def mutate(self):
        index = rd.randint(0, len(self.sentence) - 1)
        index = rd.choice([-3, -1])
        self.sentence[index] = rd.choice(corpus)  # getRandomCorpusWordIndex()

    def sentenceToStr(self):
        return ' '.join(self.sentence)


class Genetic:

    def __init__(self):
        self.size = 20
        self.steps = 100
        self.agents = []
        self.best = None

    def run(self):

        self.init()

        for i in range(self.steps):
            self.getFitness()
            self.selection()
            print(i, self.best.sentenceToStr(), self.best.fitness.item())
            self.mutate()
            self.agents[0] = cp.deepcopy(self.best)

        print(self.best.sentenceToStr())

        

    def init(self):
        for i in range(self.size):
            self.agents.append(Agent())

    def getFitness(self):
        for i in range(self.size):
            self.agents[i].getFitness()

    def selection(self):

        newAgents = []
        self.agents = sorted(self.agents, key=lambda x: x.fitness)

        self.best = cp.deepcopy(self.agents[len(self.agents) - 1])

        maxRandVal = self.size * (self.size + 1) / 2

        for i in range(self.size):
            cpt = 1
            randVal = rd.randint(0, maxRandVal - 1)
            randVal -= cpt
            while randVal > 0:
                cpt += 1
                randVal -= cpt

            newAgents.append(self.agents[cpt - 1])

        #newAgents[0] = cp.deepcopy(self.best)
        self.agents = newAgents


    def mutate(self):
        for i in range(self.size):
            self.agents[i].mutate()

gen = Genetic()
gen.run()


0 Je fais souvent ce rêve étrange et pénétrant 1.0266478061676025


IndexError: Cannot choose from an empty sequence